# Experiment A: Bound Adherence Under Nonstationary Conditions
*Question: does empirical average regret stay within the $\gamma$-regret bounds?*

### Experiment Design
Grid search over the streams (stationary, slow, abrupt, periodic), lambda levels, and two deletion ratio regimes.

Use a static comparator and calibrated learning rate schedule.
Set gamma per-theory at calibration and fix the total horizon (ie. $T=50000$).

### Primary Analysis
For each of the experimental cells, evaluate the final $R_{T}/T$ to see whether the guarantee was met.
Multiple seeds per cell in order to make some kind of causal analysis on the results.

### Success Criteria
95% of the cells meet their guarantees then I would call that a success. Which cells experience more successes than others? Can I make a heatmap that shows the regret results per cell?

## Import Statements and Versioning

This uses pretty standard library imports, but the torch requirement can stress the memory limits of a host.

In [1]:
# set global seed
import random
import numpy as np
import torch

def set_seed(seed):
    """
    Set the random seed for reproducibility.
    
    Args:
        seed (int): The seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Config and Target

In [2]:
RUN_ID = "experiment_a_test"
REGIME = "stationary"
REPLICATE = 1
T = 1000
SEED = 42
LOSS_NAME = "logistic"
MODEL_NAME = "memorypair"
LBFGS_MEM = 10

# Setup and Imports

In [3]:
# Import required modules
import sys
import os

# Add code path for imports
os.chdir(os.path.dirname(os.getcwd()))

from config import Config
from runner import ExperimentRunner

from agents.grid_runner import load_grid, generate_combinations, create_grid_id, run_parameter_combination


In [4]:
import os, json
import yaml

GRID_FILE = "grids.yaml"
BASE_OUT = "results/grid_2025_08_15"   # date-stamp it
OUTPUT_GRANULARITY = "event"           # Experiment A wants per-event for the money plot
SEEDS = list(range(5))                 # match your CLI default

# Load grid config from YAML
with open(GRID_FILE, "r") as f:
    grid_config = yaml.safe_load(f)


combos = generate_combinations(grid_config)

# select 60 combinations randomly 
cases = random.sample(combos, 10)

print(f"{len(combos)} grid cells")
print(f"sampled {len(cases)} cells to run")

# (Optional) dry-run preview
for i, p in enumerate(cases[:5]):
    print(f"{i+1:2d}. {create_grid_id(p)} → {p}")

# Make output dirs
os.makedirs(os.path.join(BASE_OUT, "sweep"), exist_ok=True)

# Run the sweep using ExperimentRunner
all_csv = []
for i, params in enumerate(cases):
    print(f"\n=== Running cell {i+1}/{len(combos)}: {create_grid_id(params)} ===")
    csvs = run_parameter_combination(
        params=params,
        seeds=SEEDS,
        base_out_dir=BASE_OUT,
        output_granularity=OUTPUT_GRANULARITY,
        parallel=1,           # bump if your runner supports safe parallelism in-notebook
    )
    all_csv.extend(csvs)

# Persist a manifest like the CLI does (handy for analysis notebooks)
manifest = { create_grid_id(p): p for p in combos }
with open(os.path.join(BASE_OUT, "sweep", "manifest.json"), "w") as f:
    json.dump(manifest, f, indent=2)

print("\nDone. CSVs:", len(all_csv))


18 grid cells
sampled 10 cells to run
 1. gamma_1.0-split_0.5_q0.95_k10_default_eps1.0_prot_ang0.01_dr0.001_fs1_hb02274d9 → {'version': 1, 'matrix': {'accountant': ['zcdp', 'eps_delta'], 'gamma_bar': [1.0], 'gamma_split': [0.3, 0.5, 0.7], 'delete_ratio': [3, 5], 'enable_oracle': [True, False], 'comparator': ['dynamic', 'static'], 'oracle_window_W': [256, 512], 'oracle_steps': [10, 15], 'oracle_stride': [None, 128], 'oracle_tol': ['1e-6'], 'oracle_warmstart': [True], 'path_length_norm': ['L2'], 'lambda_reg': [0.0, '1e-3'], 'm_max': [10], 'd_max': [None, 5.0], 'lbfgs_pair_gate_m_t': [0.0, '1e-3'], 'lbfgs_spectrum_clip': [['1e-6', '1e3']], 'ema_beta': [0.9], 'lambda_est_beta': [0.1], 'lambda_est_bounds': [['1e-8', '1e4']], 'bootstrap_iters': [500], 'quantile': [0.9], 'D_cap': [10.0], 'trim_quantile': [0.9], 'rho_total': [1.0], 'eps_total': [1.0], 'delta_total': ['1e-5'], 'delta_b': [0.05]}, 'selectors': {'name': 'lbfgs_ablation', 'where': {'lbfgs_pair_gate_m_t': [0.0, '1e-3'], 'd_max': [N